# Model Quantization and Compression Demo

In [1]:
import joblib
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import tempfile


## Load Model and Save Baseline Size

In [2]:
model_path = '../models/trained/model.joblib'
model = joblib.load(model_path)
size_before = os.path.getsize(model_path) / 1024
print(f'Model original size: {size_before:.2f} KB')

## Apply Quantization (float32 → float16)
RandomForest does not directly support quantization, but tree thresholds and values can be compressed for demo purposes.

In [3]:
def quantize_rf(rf):
    for est in rf.estimators_:
        for tree in [est.tree_]:
            tree.value[:] = tree.value.astype(np.float16)
            tree.threshold[:] = tree.threshold.astype(np.float16)
    return rf
quantized = quantize_rf(model)
with tempfile.NamedTemporaryFile(delete=False) as tmp:
    joblib.dump(quantized, tmp.name)
    size_after = os.path.getsize(tmp.name) / 1024
print(f'Compressed size: {size_after:.2f} KB')

## Evaluate Quantized Model (Optional, for demo)

In [4]:
# Load data for a quick check
import pandas as pd
df = pd.read_csv('../data/processed/features.csv')
X = df.drop(['failure_within_1yr', 'structure_id'], axis=1)
y = df['failure_within_1yr']
print('Score (original):', model.score(X, y))
print('Score (quantized):', quantized.score(X, y))